In [1]:
# pip3 install PyMySQL
import pymysql

# modules local
from modules.crawler import get_lastest_ads


In [2]:
def get_connection(DB):
    connection = pymysql.connect(
        host=DB['host'],
        port=DB['port'],
        user=DB['user'],
        passwd=DB['passwd'],
        db=DB['db']
    )

    return connection


In [3]:
def query(connection, sql):
    with connection.cursor() as cursor:
        cursor.execute(sql)
        return cursor.fetchall()


In [40]:
MYSQL = {
    'host': '127.0.0.1',
    'port': 3306,
    'user': 'root',
    'passwd': '1234',
    'db': None
}


In [44]:
connection = get_connection(MYSQL)


In [43]:
db_name = 'immoweb'
sql_create_db = "CREATE DATABASE " + db_name
query(connection, sql_create_db)


()

In [45]:
connection.close()

In [7]:
db_name = 'immoweb'
DB = {
    'host': '127.0.0.1',
    'port': 3306,
    'user': 'root',
    'passwd': '1234',
    'db': db_name
}


In [46]:
connection = get_connection(DB)


In [52]:
sql_create_table = """
CREATE TABLE IF NOT EXISTS properties (
    id INT AUTO_INCREMENT,
    immoweb_id INT NOT NULL,
    init_price INT,
    last_price INT,
    surface INT,
    rooms INT,
    zip_code INT,
    suburb VARCHAR(255),
    posted_date DATETIME,
    immoweb_link TEXT,
    type VARCHAR(255),
    PRIMARY KEY (id)
);
"""
query(connection, sql_create_table)


()

In [9]:
properties = get_lastest_ads(1)


/Users/antoinecayphas/thibaut/immoweb-teaching/modules/crawler.py:113: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 113 of the file /Users/antoinecayphas/thibaut/immoweb-teaching/modules/crawler.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  soup = BeautifulSoup(html_source)


In [53]:
columns = ('immoweb_id', 'init_price', 'last_price', 'surface', 'rooms', 'zip_code', 'suburb', 'posted_date', 'immoweb_link', 'type')

sql_insert_one_property = """
INSERT INTO `properties` (immoweb_id, init_price, last_price, surface, rooms, zip_code, suburb, posted_date, immoweb_link, type)
VALUES (%s, %s, %s, %s, %s, %s, '%s', '%s', '%s', '%s');
"""

print(sql_insert_one_property)



INSERT INTO `properties_3` (immoweb_id, init_price, last_price, surface, rooms, zip_code, suburb, posted_date, immoweb_link, type)
VALUES (%s, %s, %s, %s, %s, %s, '%s', '%s', '%s', '%s');



In [54]:

for pro in properties[0]:
    sql = sql_insert_one_property % (
        str(pro['id']),
        str(max(pro['prix']) if pro['prix'] else 'null'),
        str(min(pro['prix']) if pro['prix'] else 'null'),
        str(pro['surface'] if pro['surface'] else 'null'),
        str(pro['chambres'] if pro['chambres'] else 'null'),
        str(pro['code_postal'] if pro['code_postal'] else 'null'),
        str(pro['nom_commune']).replace("'", "''"),
        pro['date'].isoformat(),
        str(pro['lien']),
        str(pro['type'])
    )
    query(connection, sql)
connection.commit()


In [63]:
connection = get_connection(DB)

sql_select = """
    SELECT *, last_price / surface
    FROM properties_2
    WHERE last_price / surface < 2000"""

res = query(connection, sql_select)
print(res)


((1, 8182932, 139000, 139000, 91, None, 1081, 'Koekelberg', datetime.datetime(2019, 6, 13, 8, 41), 'https://www.immoweb.be/fr/annonce/commerce/a-vendre/koekelberg/1081/id8182932', 'Commerce', Decimal('1527.4725')), (4, 8182927, 139000, 139000, 70, 1, 1020, 'Bruxelles  2', datetime.datetime(2019, 6, 13, 8, 41), 'https://www.immoweb.be/fr/annonce/appartement/a-vendre/bruxelles--2/1020/id8182927', 'Appartement', Decimal('1985.7143')), (8, 8182923, 96000, 96000, 72, 2, 6960, 'Manhay', datetime.datetime(2019, 6, 13, 8, 36), 'https://www.immoweb.be/fr/annonce/appartement/a-vendre/manhay/6960/id8182923', 'Appartement', Decimal('1333.3333')), (14, 8182898, 65000, 65000, 556, None, 6780, 'Messancy', datetime.datetime(2019, 6, 13, 6, 54), 'https://www.immoweb.be/fr/annonce/terrain-a-batir/a-vendre/messancy/6780/id8182898', 'Terrain à bâtir', Decimal('116.9065')), (15, 8182893, 250000, 250000, 165, 1, 2140, 'Antwerpen Borgerhout', datetime.datetime(2019, 6, 13, 6, 53), 'https://www.immoweb.be/fr/